# Laboratorio 5

##### Javier Valle 20159
##### Mario de León 19019

#### Imports necesarios

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import multiprocessing

In [2]:
# Cargar el conjunto de datos desde un archivo CSV
dataset = pd.read_csv('mnist_train.csv')

# Extraer las etiquetas (si están presentes) y las características de las columnas del conjunto de datos
# Asumiendo que la etiqueta está en la columna "label" y las características en las columnas restantes
labels = dataset['label']
data = dataset.drop('label', axis=1)

# Normalizar los datos al rango [-1, 1] (los datos originales están en [0, 255])
data = (data.astype(np.float32) - 127.5) / 127.5

# Convertir los datos a un arreglo NumPy
data = data.values

# Definir el generador (G)
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=latent_dim, activation='relu'))
    model.add(layers.Dense(784, activation='tanh'))  # 28x28=784
    model.add(layers.Reshape((28, 28, 1)))  # Redimensionar a la forma de una imagen
    return model

# Definir el discriminador (D)
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Definir el modelo GAN que conecta el generador y el discriminador
def build_gan(generator, discriminator):
    discriminator.trainable = False  # El discriminador no se entrena cuando entrenamos el GAN
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Tamaño del espacio latente (vector de entrada para el generador)
latent_dim = 100

# Construir y compilar el generador
generator = build_generator(latent_dim)
generator.compile(loss='binary_crossentropy', optimizer='adam')

# Construir y compilar el discriminador
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Construir y compilar la GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

batch_size = 1024
epochs = 100000

for epoch in range(epochs):
    # Generar muestras falsas
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    generated_images = generator.predict(noise)

    # Seleccionar un lote aleatorio de muestras reales
    real_images = data[np.random.randint(0, data.shape[0], batch_size)]
    
    # Redimensionar las imágenes reales a (batch_size, 28, 28, 1)
    real_images = real_images.reshape((batch_size, 28, 28, 1))

    # Etiquetas para las muestras falsas y reales
    labels_real = np.ones((batch_size, 1))
    labels_fake = np.zeros((batch_size, 1))

    # Entrenar el discriminador
    d_loss_real = discriminator.train_on_batch(real_images, labels_real)
    d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Generar ruido en el espacio latente
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Etiquetas engañosas para el generador (queremos que el generador piense que las muestras son reales)
    labels_gan = np.ones((batch_size, 1))

    # Entrenar el generador a través del modelo GAN
    g_loss = gan.train_on_batch(noise, labels_gan)

    # Imprimir el progreso del entrenamiento
    
    print(f"Epoch {epoch}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

# Generar una imagen falsa al azar
noise = np.random.normal(0, 1, (1, latent_dim))
generated_image = generator.predict(noise).reshape(28, 28)

# Escalar la imagen generada al rango [0, 255] desde [-1, 1]
generated_image = (0.5 * generated_image + 0.5) * 255

# Mostrar la imagen generada
plt.imshow(generated_image, cmap='gray')
plt.axis('off')
plt.title('Imagen Generada')
plt.show()


32/32 [==============================] - 0s 1ms/step
Epoch 0/100000, D Loss: 0.8646382093429565, G Loss: 0.819901168346405
32/32 [==============================] - 0s 2ms/step
Epoch 1/100000, D Loss: 0.5854713878361508, G Loss: 0.5462244153022766
32/32 [==============================] - 0s 2ms/step
Epoch 2/100000, D Loss: 0.7535999085375806, G Loss: 0.394120454788208
32/32 [==============================] - 0s 2ms/step
Epoch 3/100000, D Loss: 0.89628380825161, G Loss: 0.2985849976539612
32/32 [==============================] - 0s 2ms/step
Epoch 4/100000, D Loss: 0.9879426740762938, G Loss: 0.26385921239852905
32/32 [==============================] - 0s 2ms/step
Epoch 5/100000, D Loss: 1.0079610446664446, G Loss: 0.2668151259422302
32/32 [==============================] - 0s 2ms/step
Epoch 6/100000, D Loss: 0.9641351239249616, G Loss: 0.3137546181678772
32/32 [==============================] - 0s 2ms/step
Epoch 7/100000, D Loss: 0.8401250787128447, G Loss: 0.43168967962265015
32/32 [===